# Libraries

In [3]:
import numpy as np

import os

import pandas as pd

# Add the ../src directory to the Python path
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

from src import (
    adjust_column_names)

# Data Import

In [5]:
source_data = pd.read_csv(
    os.path.join('..', 'data', 'insurance_data_sample.csv'),
    sep=',',
    encoding='utf-8',
    parse_dates=['Date'],
    dtype={'Phone': str})

# Preprocess

In [11]:
# prepare dataset
prep_data = source_data.copy()
prep_data = adjust_column_names(prep_data)

prep_data = prep_data\
    .groupby(['date', 'city'])\
    .agg(
        sum_of_claims=pd.NamedAgg('claim_amount', 'sum'))\
    .reset_index()

# Calculate timestamp features
prep_data.loc[:, 'day_of_week'] = prep_data.loc[:, 'date'].dt.dayofweek.astype(str)
prep_data.loc[:, 'day'] = prep_data.loc[:, 'date'].dt.day.astype(str)
prep_data.loc[:, 'month'] = prep_data.loc[:, 'date'].dt.month.astype(str)
prep_data.loc[:, 'year'] = prep_data.loc[:, 'date'].dt.year.astype(str)
prep_data.loc[:, 'yyyymm'] = prep_data.loc[:, 'date'].dt.strftime('%Y%m')

# Add time index
prep_data.loc[:, 'temp_1'] = (prep_data.loc[:, 'date'] - prep_data.loc[:, 'date'].min())
prep_data.loc[:, 'time_idx'] = (prep_data.loc[:, 'temp_1'].dt.days) + 1
prep_data.drop(['temp_1'], axis=1, inplace=True)